In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import ttk
import sqlite3
import os
import numpy as np
import scipy.io.wavfile as wav
from scipy.signal import butter, lfilter
import matplotlib.pyplot as plt

In [7]:
# ==== CLASE 1: Filtro de Kalman de Cubatura ====
class CubatureKalmanFilter:
    def __init__(self, audio_path):
        self.audio_path = audio_path

    def apply_filter(self, output_path):
        try:
            rate, data = wav.read(self.audio_path)
            if len(data.shape) > 1:  # Si es estéreo, tomar solo un canal
                data = data[:, 0]

            # Implementación del filtro de Kalman de cubatura
            def cubature_kalman_filter(data):
                # Inicialización del CKF (valores dummy para demostración)
                Q = 1e-5  # Varianza del ruido del proceso
                R = 1e-2  # Varianza del ruido de medición

                x = 0  # Estado inicial estimado
                P = 1  # Covarianza inicial estimada
                n = len(data)

                filtered_data = np.zeros(n)

                for k in range(n):
                    # Predicción
                    x_prior = x
                    P_prior = P + Q

                    # Actualización
                    K = P_prior / (P_prior + R)  # Ganancia de Kalman
                    x = x_prior + K * (data[k] - x_prior)
                    P = (1 - K) * P_prior

                    filtered_data[k] = x
                return filtered_data
            
            filtered_data = cubature_kalman_filter(data)

            # Guardar audio filtrado
            wav.write(output_path, rate, filtered_data.astype(np.int16))
            return rate, data, filtered_data

        except ValueError as e:
            print(f"Error al procesar el archivo: {e}")
            messagebox.showerror("Error", "El archivo de audio no es compatible o está dañado")
            return None, None, None

# ==== CLASE 2: Gestión de la Base de Datos ====
class DatabaseManager:
    def __init__(self, db_name="audios.db"):
        self.conn = sqlite3.connect(db_name)
        self.create_tables()

    def create_tables(self):
        with self.conn:
            self.conn.execute('''CREATE TABLE IF NOT EXISTS usuarios (
                                 id INTEGER PRIMARY KEY AUTOINCREMENT,
                                 username TEXT UNIQUE,
                                 password TEXT)''')
            self.conn.execute('''CREATE TABLE IF NOT EXISTS audios (
                                 id INTEGER PRIMARY KEY AUTOINCREMENT,
                                 user_id INTEGER,
                                 filename TEXT,
                                 filtered INTEGER,
                                 filepath TEXT,
                                 FOREIGN KEY(user_id) REFERENCES usuarios(id))''')

    def add_user(self, username, password):
        with self.conn:
            try:
                self.conn.execute("INSERT INTO usuarios (username, password) VALUES (?, ?)", (username, password))
            except sqlite3.IntegrityError:
                return False
        return True

    def validate_user(self, username, password):
        with self.conn:
            cur = self.conn.execute("SELECT id FROM usuarios WHERE username=? AND password=?", (username, password))
            return cur.fetchone()

    def add_audio(self, user_id, filename, filepath, filtered=0):
        with self.conn:
            self.conn.execute('''INSERT INTO audios (user_id, filename, filtered, filepath) 
                                 VALUES (?, ?, ?, ?)''', (user_id, filename, filtered, filepath))

    def get_audios(self, user_id, filtered=0):
        with self.conn:
            cur = self.conn.execute("SELECT id, filename, filepath FROM audios WHERE user_id=? AND filtered=?",
                                    (user_id, filtered))
            return cur.fetchall()

# ==== CLASE 3: Interfaz Gráfica (GUI) ====
class AudioFilterApp:
    def __init__(self, root):
        self.db = DatabaseManager()
        self.user_id = None

        # Configurar ventana principal
        self.root = root
        self.root.title("Audio Filter App")
        self.root.geometry("600x500")
        self.root.configure(bg="#f0f0f0")
        self.login_screen()

    def login_screen(self):
        for widget in self.root.winfo_children():
            widget.destroy()

        frame = ttk.Frame(self.root, padding="20")
        frame.pack(expand=True)

        ttk.Label(frame, text="Iniciar Sesión", font=("Helvetica", 16)).pack(pady=10)
        ttk.Label(frame, text="Usuario:", font=("Helvetica", 12)).pack(pady=5)
        self.username_entry = ttk.Entry(frame, font=("Helvetica", 12))
        self.username_entry.pack(pady=5)

        ttk.Label(frame, text="Contraseña:", font=("Helvetica", 12)).pack(pady=5)
        self.password_entry = ttk.Entry(frame, font=("Helvetica", 12), show="*")
        self.password_entry.pack(pady=5)

        ttk.Button(frame, text="Iniciar Sesión", command=self.login).pack(pady=10)
        ttk.Button(frame, text="Registrar", command=self.register).pack(pady=5)

    def login(self):
        username = self.username_entry.get()
        password = self.password_entry.get()
        user = self.db.validate_user(username, password)
        if user:
            self.user_id = user[0]
            self.main_screen()
        else:
            messagebox.showerror("Error", "Credenciales incorrectas")

    def register(self):
        username = self.username_entry.get()
        password = self.password_entry.get()
        if self.db.add_user(username, password):
            messagebox.showinfo("Éxito", "Usuario registrado")
        else:
            messagebox.showerror("Error", "El usuario ya existe")

    def cerrar_sesion(self):
        # Restablecer la sesión y regresar a la pantalla de inicio de sesión
        self.user_id = None
        self.login_screen()

    def main_screen(self):
        for widget in self.root.winfo_children():
            widget.destroy()

        frame = ttk.Frame(self.root, padding="20")
        frame.pack(expand=True)

        ttk.Label(frame, text="Bienvenido", font=("Helvetica", 16)).pack(pady=10)

        ttk.Button(frame, text="Subir Audio", command=self.upload_audio).pack(pady=10)
        ttk.Button(frame, text="Ver Audios Sin Filtrar", command=lambda: self.view_audios(filtered=0)).pack(pady=10)
        ttk.Button(frame, text="Ver Audios Filtrados", command=lambda: self.view_audios(filtered=1)).pack(pady=10)
        ttk.Button(frame, text="Cerar Sesion", command=self.login_screen).pack(pady=10)
    def validate_audio(self, filepath):
        try:
            rate, data = wav.read(filepath)
            return True
        except ValueError:
            return False

    def upload_audio(self):
        file_path = filedialog.askopenfilename(filetypes=[("Audio Files", "*.wav")])
        if file_path:
            if not self.validate_audio(file_path):
                messagebox.showerror("Error", "El archivo seleccionado no es un archivo WAV válido")
                return
            filename = os.path.basename(file_path)
            self.db.add_audio(self.user_id, filename, file_path)
            self.auto_process_audio(file_path)

    def auto_process_audio(self, filepath):
        output_path = "filtered_" + os.path.basename(filepath)
        kalman_filter = CubatureKalmanFilter(filepath)
        rate, original_data, filtered_data = kalman_filter.apply_filter(output_path)

        self.db.add_audio(self.user_id, os.path.basename(output_path), output_path, filtered=1)

        # Visualizar resultados
        if rate is not None:
            self.plot_audio(rate, original_data, filtered_data)
            messagebox.showinfo("Éxito", "Audio filtrado y guardado")

    def plot_audio(self, rate, original_data, filtered_data):
        plt.figure(figsize=(10, 6))

        plt.subplot(2, 1, 1)
        plt.plot(original_data, color='blue', label='Original')
        plt.title("Audio Original")
        plt.xlabel("Muestras")
        plt.ylabel("Amplitud")
        plt.legend()

        plt.subplot(2, 1, 2)
        plt.plot(filtered_data, color='green', label='Filtrado')
        plt.title("Audio Filtrado")
        plt.xlabel("Muestras")
        plt.ylabel("Amplitud")
        plt.legend()

        plt.tight_layout()
        plt.show()

    def view_audios(self, filtered):
        for widget in self.root.winfo_children():
            widget.destroy()

        frame = ttk.Frame(self.root, padding="20")
        frame.pack(expand=True)

        ttk.Label(frame, text="Audios", font=("Helvetica", 16)).pack(pady=10)

        audios = self.db.get_audios(self.user_id, filtered)
        print("Estos son los audios: ", audios)
        for audio in audios:
            audio_id, filename, filepath = audio
            if filtered==1 and "filtered" in filename:
                print("Audio filtrado")
                ttk.Button(frame, text=filename, 
                        command=lambda fp=filepath: self.show_audio_data(fp)).pack(pady=5)
            
            if filtered==0 and "filtered" not in filename:
                print("Audio filtrado")
                ttk.Button(frame, text=filename, 
                        command=lambda fp=filepath: self.show_audio_data(fp)).pack(pady=5)
                
        ttk.Button(frame, text="Volver", command=self.main_screen).pack(pady=10)

    def show_audio_data(self, filepath):
        kalman_filter = CubatureKalmanFilter(filepath)
        rate, original_data, filtered_data = kalman_filter.apply_filter("temp_filtered.wav")

        if rate is not None:
            self.plot_audio(rate, original_data, filtered_data)

        messagebox.showinfo("Info", "Datos del audio mostrados")

# ==== EJECUCIÓN DEL PROGRAMA ====
if __name__ == "__main__":
    root = tk.Tk()
    app = AudioFilterApp(root)
    root.mainloop()
